# Scipy - scientific library for Python


`scipy` is built on top of the `numpy` framework for multidimensional arrays, and provides a large number of higher-level scientific algorithms. Some of the topics that SciPy covers are:

* Special functions ([scipy.special](http://docs.scipy.org/doc/scipy/reference/special.html))
* Integration ([scipy.integrate](http://docs.scipy.org/doc/scipy/reference/integrate.html))
* Optimization ([scipy.optimize](http://docs.scipy.org/doc/scipy/reference/optimize.html))
* Interpolation ([scipy.interpolate](http://docs.scipy.org/doc/scipy/reference/interpolate.html))
* Fourier Transforms ([scipy.fftpack](http://docs.scipy.org/doc/scipy/reference/fftpack.html))
* Signal Processing ([scipy.signal](http://docs.scipy.org/doc/scipy/reference/signal.html))
* Linear Algebra ([scipy.linalg](http://docs.scipy.org/doc/scipy/reference/linalg.html))
* Sparse Eigenvalue Problems ([scipy.sparse](http://docs.scipy.org/doc/scipy/reference/sparse.html))
* Statistics ([scipy.stats](http://docs.scipy.org/doc/scipy/reference/stats.html))
* Multi-dimensional image processing ([scipy.ndimage](http://docs.scipy.org/doc/scipy/reference/ndimage.html))
* File IO ([scipy.io](http://docs.scipy.org/doc/scipy/reference/io.html))

Each of these submodules provides a number of functions and classes that can be used to solve problems in their respective topics.


## Special functions
Scipy implements a large amount of *special functions* (Bessel function,
Airy function, orthogonal polynomials, ...) for numneric calculations. They can be used as functions within `numpy`.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.special as ss

# we plot the n\in [1..3] Legendre polynomials.
#
x = np.linspace(-1.0, 1.0, 100)

for n in range(1, 4):
    y = ss.eval_legendre(n, x)
    plt.plot(x, y, label=r"$L_%d(x)$" % n)
    
plt.legend()    

## Optimization / Fitting

Determining a parametric model $y = m(x; a_0, a_1, \dots a_n)$, where the $a_i$ are parameters we would like to determine) to given data points
$(x_i; y_i \pm \Delta y_i);\; i\in [1, \dots, m]$ is called *data-fitting*. Usually the measurements $y_i$ come with some errors $\Delta y_i$. `Scipy` offers several functions for data fitting and I will show you the simplest one: `curve_fit`. It determines the best fit parameters with the $\chi^2$-method, i.e. it determines best fit parameters by minimizing the expression:

$$
\chi^2 = \sum_{i=1}^n\frac{(y_i-m(x_i; a_0, a_1, \dots a_n))^2}{(\Delta y_i)^2}
$$

Please read the [curve_fit documentation](http://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html#scipy.optimize.curve_fit) on details (error handling etc.).

For demonstration purposes we perform a line fit on some fake data:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as nr
import scipy.optimize as so

# create some fake data and plot them:
numpoints = 10 # the number of points we want to simulate
t = np.random.uniform(0., 100., numpoints)
# The error on each point comes from a normal distribution
# with sigma = 10
s = 2. * t + 2.6 + np.random.normal(0., 10., numpoints)
plt.errorbar(t, s, yerr=10, fmt=".")

In [ ]:
def fit_line(t, v, s0):
    return v * t + s0

# now perform the fit
# Please read carefully the documentation to see how errors
# are handled. In Physics we typically give absolute errors,
# not relative ones!
popt, pcov = so.curve_fit(fit_line, t, s,
                          sigma = np.ones(y.shape) * 10, absolute_sigma=True)
print(popt, pcov)
print(f"v = {popt[0]} +/- {np.sqrt(pcov[0][0])}")
print(f"s0 = {popt[1]} +/- {np.sqrt(pcov[1][1])}")

t_fit = np.linspace(0.0, 100, 100)
s_fit = fit_line(t_fit, *(popt))
plt.errorbar(t, s, yerr=10, fmt=".")
plt.plot(t_fit, s_fit, 'r-')

### Careful when the fit parameters seem fine but not the errors

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as nr
import scipy.optimize as so

def fit_func(x, c):
    return c * np.exp(x)

numpoints = 10 # the number of points we want to simulate
x = np.random.uniform(-2., 2., numpoints)
y = 0.2 * np.exp(x + 5) + np.random.normal(0., 10., numpoints)
plt.errorbar(x, y, yerr=10, fmt=".")

popt, pcov = so.curve_fit(fit_func, x, y,
                          sigma = np.ones(y.shape) * 10, absolute_sigma=True)
print(popt, pcov)
x_fit = np.linspace(-2.0, 2.0, 20)
plt.plot(x_fit, fit_func(x_fit, *(popt)), 'r-')

### Warning: Do not use `curve_fit` if you have errors in both variables

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as nr
import scipy.optimize as so

def fit_line(x, a, b):
    return a * x + b

# create some fake data with errors in 'both' coordinates and plot them:
numpoints = 50 # the number of points we want to simulate
x = np.random.uniform(0., 100., numpoints)
y = x

# add errors in x and y
x = x + np.random.normal(0., 10., numpoints)
y = y + np.random.normal(0., 10., numpoints)

plt.errorbar(x, y, xerr=10, yerr=10, fmt=".")

# perform a fit as we did before (neglecting errors in x!):
popt, pcov = so.curve_fit(fit_line, x, y,
                          sigma = np.ones(y.shape) * 10, absolute_sigma=True)
plt.plot(x, fit_line(x, *(popt)), 'r-')
print(popt, np.sqrt(pcov[0,0]))

# now change the roles of x and y and fit again:
x1, y1 = y, x
popt, pcov = so.curve_fit(fit_line, x1, y1,
                          sigma = np.ones(y.shape) * 10, absolute_sigma=True)
plt.plot(x1, fit_line(x1, *(popt)), 'g-')
print(popt)
# Note that you do not get the 'inverse' of the first fit as you might expect!

For fits with errors in $x$ and $y$ you have to use other methods such as *Orthogonal distance regression*. I give you an example script with [chi2FitXYErr.py](code/chi2FitXYErr.py).

In [ ]:
!cat ./data/dataxy.txt
!cat ./data/dataxy_reversed.txt

In [ ]:
%run ./code/chi2FitXYErr.py -i ./data/dataxy.txt

In [ ]:
# The fit-result from the reversed data set also gives a 'inverse' line!
%run ./code/chi2FitXYErr.py -i ./data/dataxy_reversed.txt

## Another view on the linear fitting problem (relevant for Project 3)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as so

# we want to estimate parameters s0 and v from
# s = s0 + v * t
def car(t, s0, v):
    return s0 + v * t

t = np.array([1, 2, 3, 4])
s = np.array([1, 2, 2, 5])

plt.plot(t, s, marker="o", linestyle="")

popt, pcov = so.curve_fit(car, t, s, absolute_sigma=True)
print(popt, pcov)

t_plot = np.linspace(0.0, 5.0, 10)
s_plot = car(t_plot, *popt)

plt.plot(t_plot, s_plot)